# openGauss VectorStore

本 Notebook 将介绍 openGauss VectorStore 的入门方法。[openGauss](https://opengauss.org/en/) 是一个高性能的关系型数据库，具备原生的向量存储和检索能力。此次集成使得 LangChain 应用程序内的向量操作符合 ACID 标准，将传统的 SQL 功能与现代的 AI 驱动的相似性搜索相结合。
 vector store。

## 设置

### 启动 openGauss 容器

```bash
docker run --name opengauss \
  -d \
  -e GS_PASSWORD='MyStrongPass@123' \
  -p 8888:5432 \
  opengauss/opengauss-server:latest
```

### 安装 langchain-opengauss

**系统要求**:
- openGauss ≥ 7.0.0
- Python ≥ 3.8
- psycopg2-binary

### 凭据

使用你的 openGauss 凭据

## 初始化

import EmbeddingTabs from "@theme/EmbeddingTabs";

<EmbeddingTabs/>

In [ ]:
from langchain_opengauss import OpenGauss, OpenGaussSettings

# Configure with schema validation
config = OpenGaussSettings(
    table_name="test_langchain",
    embedding_dimension=384,
    index_type="HNSW",
    distance_strategy="COSINE",
)
vector_store = OpenGauss(embedding=embeddings, config=config)

## 管理向量存储

### 向向量存储添加条目

In [ ]:
from langchain_core.documents import Document

document_1 = Document(page_content="foo", metadata={"source": "https://example.com"})

document_2 = Document(page_content="bar", metadata={"source": "https://example.com"})

document_3 = Document(page_content="baz", metadata={"source": "https://example.com"})

documents = [document_1, document_2, document_3]

vector_store.add_documents(documents=documents, ids=["1", "2", "3"])

### 更新向量存储中的条目

In [ ]:
updated_document = Document(
    page_content="qux", metadata={"source": "https://another-example.com"}
)

# If the id is already exist, will update the document
vector_store.add_documents(document_id="1", document=updated_document)

### 从向量存储中删除项目

In [ ]:
vector_store.delete(ids=["3"])

## 查询向量存储

创建向量存储并添加相关文档后，你最有可能希望在链或代理运行时查询它。

### 直接查询

执行简单的相似性搜索可以这样做：

- TODO: 编辑并运行代码单元以生成输出

In [ ]:
results = vector_store.similarity_search(
    query="thud", k=1, filter={"source": "https://another-example.com"}
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

如果你想执行相似性搜索并获得相应的得分，可以运行：

In [ ]:
results = vector_store.similarity_search_with_score(
    query="thud", k=1, filter={"source": "https://example.com"}
)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

### 通过转换向量存储为检索器

你也可以将向量存储转换为检索器，以便在你的链中使用。

- TODO: 编辑后运行代码单元以生成输出

In [ ]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("thud")

## 用于检索增强生成的使用方法

有关如何将此向量存储用于检索增强生成 (RAG) 的指南，请参阅以下章节：

- [教程](/docs/tutorials/)
- [操作方法：使用 RAG 进行问答](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [检索概念文档](https://python.langchain.com/docs/concepts/retrieval/)

## 配置

### 连接设置
| 参数           | 默认值                 | 描述                                                                                                                                                                                                                                                                                          |
|-----------------|-------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `host`          | localhost               | 数据库服务器地址                                                                                                                                                                                                                                                                                              |
| `port`          | 8888                    | 数据库连接端口                                                                                                                                                                                                                                                                                                |
| `user`          | gaussdb                 | 数据库用户名                                                                                                                                                                                                                                                                                                  |
| `password`      | -                       | 复杂密码字符串                                                                                                                                                                                                                                                                                              |
| `database`      | postgres                | 默认数据库名称                                                                                                                                                                                                                                                                                                |
| `min_connections` | 1                       | 连接池最小大小                                                                                                                                                                                                                                                                                                |
| `max_connections` | 5                       | 连接池最大大小                                                                                                                                                                                                                                                                                                |
| `table_name`    | langchain_docs          | 用于存储向量数据和元数据的表名                                                                                                                                                                                                                                                                                |
| `index_type`    | IndexType.HNSW          | 向量索引算法类型。选项：HNSW 或 IVFFLAT\n默认值为 HNSW。                                                                                                                                                                                                                                                        |
| `vector_type`   | VectorType.vector       | 要使用的向量表示类型。默认值为 Vector。                                                                                                                                                                                                                                                                       |
| `distance_strategy` | DistanceStrategy.COSINE | 用于检索的向量相似性度量。选项：euclidean（L2 距离）、cosine（角度距离，对文本嵌入效果最佳）、manhattan（L1 距离，适用于稀疏数据）、negative_inner_product（归一化向量的点积）。\n默认值为 cosine。                                                                                                                                                                                     |
|`embedding_dimension`| 1536                    | 向量嵌入的维度。                                                                                                                                                                                                                                                                                              |

### 支持的组合

| 向量类型 | 维度     | 索引类型  | 支持的距离策略                               |
|---------|----------|----------|--------------------------------------------|
| vector  | ≤2000    | HNSW/IVFFLAT | COSINE/EUCLIDEAN/MANHATTAN/INNER_PROD        |

## 性能优化

### 索引调优指南
**HNSW 参数**:
- `m`: 16-100 (召回率和内存的平衡)
- `ef_construction`: 64-1000 (必须大于 2*m)

**IVFFLAT 建议**:
```python
import math

lists = min(
    int(math.sqrt(total_rows)) if total_rows > 1e6 else int(total_rows / 1000),
    2000,  # openGauss maximum
)
```

### 连接池
```python
OpenGaussSettings(min_connections=3, max_connections=20)
```

## 限制
- `bit` 和 `sparsevec` 向量类型目前正在开发中
- 最大向量维度：`vector` 类型为 2000

## API 参考

有关 __ModuleName__VectorStore 所有功能和配置的详细文档，请访问 API 参考：https://python.langchain.com/api_reference/en/latest/vectorstores/opengauss.OpenGuass.html